# Lab 2: Add Memory to Investment Agents

This notebook enhances the 2-agent investment research system from Lab 1 with Amazon Bedrock AgentCore Memory for learning investor preferences and cross-session continuity.

## Overview

Building on Lab 1, we'll add:
- **AgentCore Memory Resources**: USER_PREFERENCE and SEMANTIC strategies
- **Memory Hooks**: Retrieve context and store interactions
- **Cross-Session Persistence**: SSM Parameter Store for memory_id
- **Personalized Analysis**: Memory-aware investment recommendations

This follows the **Lab 2** pattern from 07-AgentCore-E2E: enhance local agents with memory capabilities before runtime deployment (Lab 3).

## Prerequisites

- **Lab 1 completed**: Local agents created and tested
- Python 3.10+
- AWS credentials configured
- Amazon Bedrock AgentCore access
- Required Python packages (see requirements.txt)

## Step 1: Environment Setup and Authentication

Set up the environment with AWS credentials and required libraries.

In [ ]:
# Install required packages from requirements.txt
!pip install -q -r requirements.txt

In [ ]:
# Import required libraries
import os
import boto3
import json
from datetime import datetime
from IPython.display import display, Markdown

# Strands imports
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import think

# AgentCore Memory imports
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType
# Note: HookProvider and events may not be available in current version
# We'll implement memory functionality without hooks for now

print("✅ All libraries imported successfully")

In [ ]:
# Set AWS credentials - Update these with your temporary credentials only for local machine
os.environ['AWS_ACCESS_KEY_ID'] = 'AWS_ACCESS_KEY_ID'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'AWS_SECRET_ACCESS_KEY'
os.environ['AWS_SESSION_TOKEN'] = 'AWS_SESSION_TOKEN'
os.environ['AWS_DEFAULT_REGION'] = 'us-west-2'

print("AWS credentials set successfully!")

# Clear boto3 cached sessions
boto3.DEFAULT_SESSION = None

# Verify credentials and extract account ID
try:
    sts = boto3.client('sts')
    identity = sts.get_caller_identity()
    AWS_ACCOUNT_ID = identity['Account']
    AWS_REGION = os.environ['AWS_DEFAULT_REGION']
    
    print(f"✅ Using AWS Account: {AWS_ACCOUNT_ID}")
    print(f"✅ User/Role: {identity['Arn']}")
    print(f"✅ Region: {AWS_REGION}")
except Exception as e:
    print(f"❌ Credential verification failed: {e}")
    raise

## Step 2: Create Memory Utilities

Create utility functions for SSM Parameter Store to persist memory_id across sessions.

In [ ]:
%%writefile lab_helpers/utils.py
import boto3
from botocore.exceptions import ClientError

def get_ssm_parameter(parameter_name: str) -> str:
    """Get parameter value from SSM Parameter Store."""
    ssm = boto3.client('ssm')
    try:
        response = ssm.get_parameter(Name=parameter_name, WithDecryption=True)
        return response['Parameter']['Value']
    except ClientError as e:
        if e.response['Error']['Code'] == 'ParameterNotFound':
            raise ValueError(f"Parameter {parameter_name} not found")
        else:
            raise e

def put_ssm_parameter(parameter_name: str, parameter_value: str, description: str = "") -> None:
    """Store parameter value in SSM Parameter Store."""
    ssm = boto3.client('ssm')
    try:
        ssm.put_parameter(
            Name=parameter_name,
            Value=parameter_value,
            Type='String',
            Description=description,
            Overwrite=True
        )
    except ClientError as e:
        raise e

print("✅ Memory utilities functions defined")

## Step 2.5: Optional IAM Permissions Setup

**Note**: These cells are optional and only needed if running in SageMaker/Jupyter.

If you get SSM or Bedrock access denied errors, run these cells to add the necessary permissions to your execution role.

**Skip these cells if running locally with AWS credentials.**

In [ ]:
# OPTIONAL: Add SSM permissions to SageMaker/execution role
# Only run this if you get SSM GetParameter/PutParameter access denied errors

import boto3
import json
import time

# Get current role and account info
sts = boto3.client('sts')
identity = sts.get_caller_identity()
role_arn = identity['Arn']
role_name = role_arn.split('/')[-2]  # Extract role name
account_id = identity['Account']
region = boto3.Session().region_name or 'us-west-2'

print(f"Account ID: {account_id}")
print(f"Region: {region}")
print(f"Execution Role: {role_name}")

# Add SSM permissions
iam = boto3.client('iam')

ssm_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "ssm:GetParameter",
                "ssm:GetParameters",
                "ssm:PutParameter"
            ],
            "Resource": f"arn:aws:ssm:{region}:{account_id}:parameter/app/investment/agentcore/*"
        }
    ]
}

try:
    iam.put_role_policy(
        RoleName=role_name,
        PolicyName="InvestmentAgentSSMAccess",
        PolicyDocument=json.dumps(ssm_policy)
    )
    print(f"✅ Added SSM permissions to {role_name}")
    print(f"   Resource: arn:aws:ssm:{region}:{account_id}:parameter/app/investment/agentcore/*")
    
    # Wait for IAM to propagate
    print("\n⏳ Waiting 15 seconds for IAM permissions to propagate...")
    for i in range(15, 0, -3):
        print(f"   {i} seconds remaining...")
        time.sleep(3)
    print("✅ Ready to proceed!")
    print("\n⚠️ Note: If you still get errors, restart the kernel (Kernel → Restart) and run from the beginning.")
    
except Exception as e:
    print(f"❌ Error adding permissions: {e}")
    print("\nYou may need to add these permissions manually in the IAM Console.")

In [ ]:
# OPTIONAL: Add Bedrock model access permissions
# Only run this if you get Bedrock InvokeModel access denied errors

import boto3
import json
import time

# Get current role
sts = boto3.client('sts')
identity = sts.get_caller_identity()
role_name = identity['Arn'].split('/')[-2]

print(f"Adding Bedrock permissions to: {role_name}")

# Add Bedrock permissions
iam = boto3.client('iam')

bedrock_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:InvokeModel",
                "bedrock:InvokeModelWithResponseStream"
            ],
            "Resource": [
                "arn:aws:bedrock:*::foundation-model/us.amazon.nova-pro-v1:0",
                "arn:aws:bedrock:*::foundation-model/*"
            ]
        }
    ]
}

try:
    iam.put_role_policy(
        RoleName=role_name,
        PolicyName="BedrockModelAccess",
        PolicyDocument=json.dumps(bedrock_policy)
    )
    print("✅ Added Bedrock model access permissions")
    
    print("\n⏳ Waiting 10 seconds for permissions to propagate...")
    time.sleep(10)
    print("✅ Ready!")
    print("\n⚠️ Note: If you still get errors, restart the kernel (Kernel → Restart) and run from the beginning.")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("\nYou may need to add these permissions manually in the IAM Console.")

## Step 3: Create AgentCore Memory Resources

Create or retrieve existing memory resource with USER_PREFERENCE and SEMANTIC strategies.

In [ ]:
# Import utilities
from lab_helpers.utils import get_ssm_parameter, put_ssm_parameter

# Initialize memory client
memory_client = MemoryClient(region_name=AWS_REGION)

# Define memory strategies
strategies = [
    {
        StrategyType.USER_PREFERENCE.value: {
            "name": "InvestorPreferences",
            "description": "Captures investor preferences, risk tolerance, and investment patterns",
            "namespaces": ["investment/investor/{actorId}/preferences"],
        }
    },
    {
        StrategyType.SEMANTIC.value: {
            "name": "InvestmentSemantic", 
            "description": "Stores investment analysis and market insights",
            "namespaces": ["investment/investor/{actorId}/semantic"],
        }
    },
]

def create_or_get_memory_resource():
    """Create new memory resource or retrieve existing one using SSM Parameter Store."""
    memory_param_name = "/app/investment/agentcore/memory_id"
    
    try:
        # Try to get existing memory_id from SSM
        memory_id = get_ssm_parameter(memory_param_name)
        print(f"✅ Found existing memory resource: {memory_id}")
        
        # Verify memory still exists
        try:
            memory_info = memory_client.get_memory(memoryId=memory_id)
            print(f"✅ Memory resource verified: {memory_info.get('name', memory_id)}")
            return memory_id
        except Exception as e:
            print(f"⚠️ Existing memory not accessible: {e}")
            print("Creating new memory resource...")
            
    except ValueError:
        print("📝 No existing memory found. Creating new memory resource...")
    
    # Create new memory resource with unique name
    try:
        import uuid
        unique_suffix = str(uuid.uuid4())[:8]
        memory_name = f"InvestmentResearchMemory_{unique_suffix}"
        
        memory = memory_client.create_memory_and_wait(
            name=memory_name,
            description="Investment research agent memory for learning investor preferences and analysis history",
            strategies=strategies,
            event_expiry_days=90,
        )
        
        memory_id = memory['id']
        
        # Store memory_id in SSM for future sessions
        put_ssm_parameter(
            memory_param_name, 
            memory_id,
            "Investment Research Agent Memory ID for cross-session persistence"
        )
        
        print(f"✅ Created new memory resource: {memory_id}")
        print(f"✅ Memory ID stored in SSM: {memory_param_name}")
        return memory_id
        
    except Exception as e:
        print(f"❌ Failed to create memory resource: {e}")
        raise

# Create or get memory resource
MEMORY_ID = create_or_get_memory_resource()
print(f"\n🧠 Memory Resource Ready: {MEMORY_ID}")

## Step 4: Create Memory Hooks

Implement memory hooks to retrieve investor context and store interaction details.

In [ ]:
%%writefile lab_helpers/investment_memory_hooks.py
import json
from typing import Dict, Any
from bedrock_agentcore.memory import MemoryClient
from strands.hooks import HookProvider, HookRegistry, MessageAddedEvent, AfterInvocationEvent

class InvestmentMemoryHooks(HookProvider):
    """Memory hooks for investment research agent to learn investor preferences."""
    
    def __init__(self, memory_id: str, memory_client: MemoryClient, actor_id: str, session_id: str):
        self.memory_id = memory_id
        self.memory_client = memory_client
        self.actor_id = actor_id
        self.session_id = session_id
    
    def retrieve_investor_context(self, event: MessageAddedEvent) -> None:
        """Retrieve investor context and preferences before processing query."""
        messages = event.agent.messages
        if messages[-1]["role"] == "user" and "toolResult" not in messages[-1]["content"][0]:
            user_query = messages[-1]["content"][0]["text"]
            
            try:
                all_context = []
                
                # Retrieve from preferences namespace
                pref_memories = self.memory_client.retrieve_memories(
                    memory_id=self.memory_id,
                    namespace=f"investment/investor/{self.actor_id}/preferences",
                    query=user_query,
                    top_k=3
                )
                
                # Retrieve from semantic namespace
                sem_memories = self.memory_client.retrieve_memories(
                    memory_id=self.memory_id,
                    namespace=f"investment/investor/{self.actor_id}/semantic",
                    query=user_query,
                    top_k=3
                )
                
                # Format context from both namespaces
                for memory in pref_memories:
                    if isinstance(memory, dict):
                        content = memory.get('content', {})
                        if isinstance(content, dict):
                            text = content.get('text', '').strip()
                            if text:
                                all_context.append(f"[PREFERENCE] {text}")
                
                for memory in sem_memories:
                    if isinstance(memory, dict):
                        content = memory.get('content', {})
                        if isinstance(content, dict):
                            text = content.get('text', '').strip()
                            if text:
                                all_context.append(f"[SEMANTIC] {text}")
                
                # Inject context into the message
                if all_context:
                    context_text = "\n".join(all_context)
                    original_text = messages[-1]["content"][0]["text"]
                    messages[-1]["content"][0]["text"] = f"Investor Context:\n{context_text}\n\n{original_text}"
                    print(f"🧠 Retrieved {len(all_context)} context items from memory")
                else:
                    print("📝 No relevant context found - first interaction or new topic")
                
            except Exception as e:
                print(f"⚠️ Error retrieving investor context: {e}")
                # Continue without context rather than failing
    
    def save_investment_interaction(self, event: AfterInvocationEvent) -> None:
        """Save complete investment interaction details after response generation."""
        try:
            messages = event.agent.messages
            if len(messages) >= 2 and messages[-1]["role"] == "assistant":
                # Get last user query and agent response
                user_query = None
                agent_response = None
                
                for msg in reversed(messages):
                    if msg["role"] == "assistant" and not agent_response:
                        agent_response = msg["content"][0]["text"]
                    elif msg["role"] == "user" and not user_query and "toolResult" not in msg["content"][0]:
                        user_query = msg["content"][0]["text"]
                        break
                
                if user_query and agent_response:
                    # Store interaction in memory
                    self.memory_client.create_event(
                        memory_id=self.memory_id,
                        actor_id=self.actor_id,
                        session_id=self.session_id,
                        messages=[
                            (user_query, "USER"),
                            (agent_response, "ASSISTANT"),
                        ],
                    )
                    print(f"💾 Saved interaction to memory: {len(user_query)} chars query, {len(agent_response)} chars response")
            
        except Exception as e:
            print(f"⚠️ Error saving interaction to memory: {e}")
            # Continue without saving rather than failing

    def register_hooks(self, registry: HookRegistry) -> None:
        """Register investment memory hooks"""
        registry.add_callback(MessageAddedEvent, self.retrieve_investor_context)
        registry.add_callback(AfterInvocationEvent, self.save_investment_interaction)
        print("Investment memory hooks registered")

print("✅ Investment Memory Hooks defined")

## Step 5: Create Memory-Enhanced Supervisor Agent

Update the supervisor agent with memory hooks for personalized investment analysis.

In [ ]:
%%writefile lab_helpers/lab2_supervisor_with_memory.py
import json
from typing import Dict, Any, List
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import think

# Import the QA agent from Lab 1
from lab_helpers.lab1_quantitative_agent import create_quantitative_analysis_agent
from lab_helpers.investment_memory_hooks import InvestmentMemoryHooks

@tool
def quantitative_analysis(query: str) -> str:
    """Delegate quantitative analysis tasks to the specialized QA agent."""
    try:
        qa_agent = create_quantitative_analysis_agent()
        response = qa_agent(query)
        return response.message["content"][0]["text"]
    except Exception as e:
        return f"Error in quantitative analysis: {str(e)}"

def create_memory_enhanced_investment_assistant(memory_id: str, actor_id: str = "investor_001"):
    """Create investment research assistant with memory capabilities."""
    
    # Create memory hooks
    import uuid
    from bedrock_agentcore.memory import MemoryClient
    import os
    
    session_id = str(uuid.uuid4())
    memory_client = MemoryClient(region_name=os.environ.get('AWS_DEFAULT_REGION', 'us-west-2'))
    memory_hooks = InvestmentMemoryHooks(memory_id, memory_client, actor_id, session_id)
    
    # Create agent with enhanced system prompt for memory awareness
    agent = Agent(
        name="memory_enhanced_investment_assistant",
        system_prompt="""You are a Memory-Enhanced Investment Research Assistant, a financial research supervisor with learning capabilities.

Your enhanced responsibilities:
1. Analyze user queries and leverage previous interaction context when available
2. Delegate quantitative analysis tasks to the quantitative_analysis tool
3. Synthesize analysis with personalized insights based on learned investor preferences
4. Generate investment recommendations that adapt to investor's risk tolerance and interests
5. Build comprehensive investor profiles over time through interaction learning

Memory-Aware Workflow:
1. Review any provided INVESTOR CONTEXT from previous sessions
2. Use quantitative_analysis tool for current market data and analysis
3. Combine current analysis with learned preferences for personalized recommendations
4. Provide investment advice that considers the investor's historical interests and risk profile

Personalization Guidelines:
- Reference previous stock interests when relevant
- Adapt analysis depth to investor's demonstrated expertise level
- Consider previously expressed risk tolerance in recommendations
- Build on previous investment themes and sector preferences
- Acknowledge learning progression: "Based on your previous interest in..."

Always provide comprehensive, personalized investment analysis that demonstrates learning from previous interactions while incorporating current quantitative data.""",
        model=BedrockModel(model_id="us.amazon.nova-pro-v1:0", region_name="us-west-2"),
        tools=[quantitative_analysis, think],
        hooks=[memory_hooks]
    )
    
    return agent

print("✅ Memory-Enhanced Investment Research Assistant functions defined")

## Step 6: Test Memory-Enhanced System

Test the memory functionality with the enhanced supervisor agent.

In [ ]:
# Import the memory-enhanced supervisor
from lab_helpers.lab2_supervisor_with_memory import create_memory_enhanced_investment_assistant

# Create memory-enhanced agent
memory_agent = create_memory_enhanced_investment_assistant(memory_id=MEMORY_ID)

print("✅ Memory-Enhanced Investment Research Assistant created successfully")
print(f"Agent name: {memory_agent.name}")
print(f"Memory ID: {MEMORY_ID}")
print("✅ Agent configured with memory hooks and quantitative analysis delegation")

In [ ]:
# Session 1: Initial interaction - establish baseline preferences
print("🧪 Session 1: Initial Investment Analysis (Building Investor Profile)\n")

session_1_query = """I'm interested in analyzing Apple (AAPL) stock. 
I'm generally a moderate risk investor focused on long-term growth. 
Please provide a comprehensive analysis."""

response_1 = memory_agent(session_1_query)

print("📊 Session 1 Response:")
display(Markdown(response_1.message["content"][0]["text"]))
print("\n💾 This interaction has been stored in memory for future personalization.")

In [ ]:
# Session 2: Follow-up query - test context retrieval
print("🧪 Session 2: Follow-up Analysis (Testing Memory Retrieval)\n")

session_2_query = "How is AAPL performing now compared to when we last discussed it?"

response_2 = memory_agent(session_2_query)

print("📈 Session 2 Response:")
display(Markdown(response_2.message["content"][0]["text"]))
print("\n🧠 Notice how the agent references previous context and provides comparative analysis.")

In [ ]:
# Session 3: Personalized recommendations - test learned preferences
print("🧪 Session 3: Personalized Recommendations (Testing Learned Preferences)\n")

session_3_query = "Based on my investment profile, can you recommend some other stocks I might be interested in?"

response_3 = memory_agent(session_3_query)

print("🎯 Session 3 Response:")
display(Markdown(response_3.message["content"][0]["text"]))
print("\n✨ Notice how recommendations are tailored to your previously expressed preferences and risk tolerance.")

## Step 7: Test Cross-Session Memory Persistence

Simulate a new session by creating a fresh agent instance to test memory persistence.

In [ ]:
# Simulate new session by creating fresh agent instance
print("🔄 Testing Cross-Session Memory Persistence\n")
print("Creating fresh agent instance (simulating notebook restart)...")

# Create new agent instance with same memory_id
fresh_memory_agent = create_memory_enhanced_investment_assistant(memory_id=MEMORY_ID)

print("✅ Fresh agent created with existing memory resource")
print(f"Memory ID: {MEMORY_ID}")

# Test with query that should trigger memory retrieval
cross_session_query = "What stocks have we discussed before, and what were my key preferences?"

response_cross = fresh_memory_agent(cross_session_query)

print("\n🧠 Cross-Session Memory Test Response:")
display(Markdown(response_cross.message["content"][0]["text"]))
print("\n✅ If memory is working correctly, the agent should reference previous AAPL discussions and your moderate risk, long-term growth preferences.")

## Step 8: Memory Validation and Summary

Validate memory functionality and summarize Lab 2 accomplishments.

In [ ]:
# Memory validation summary
print("📋 Lab 2 Memory Validation Summary")
print("=" * 50)

# Check memory resource
try:
    memory_info = memory_client.get_memory(MEMORY_ID)
    memory_exists = True
    memory_name = memory_info.get('name', 'Unknown')
except:
    memory_exists = False
    memory_name = 'Not accessible'

# Check SSM parameter
try:
    stored_memory_id = get_ssm_parameter("/app/investment/agentcore/memory_id")
    ssm_working = stored_memory_id == MEMORY_ID
except:
    ssm_working = False

# Check agent functionality
try:
    test_response = fresh_memory_agent("Quick test")
    agent_working = True
except:
    agent_working = False

print(f"✅ Memory Resource Created: {memory_exists}")
print(f"✅ Memory Name: {memory_name}")
print(f"✅ Memory ID: {MEMORY_ID}")
print(f"✅ SSM Parameter Store: {ssm_working}")
print(f"✅ Memory-Enhanced Agent: {agent_working}")
print(f"✅ Cross-Session Persistence: {memory_exists and ssm_working}")